# contoso purview 

https://ms.portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/8c2c7b23-848d-40fe-b817-690d79ad9dfd/resourceGroups/contosoPurview/providers/Microsoft.Purview/accounts/ContosoPurview/overview

from amjad: 
https://ms.portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/855baa9a-294b-48cd-88fe-89e90ab90ed9/resourceGroups/myResourceGroup/providers/Microsoft.Purview/accounts/purviewdemoamk/overview

## REST APIs for daa planes

https://learn.microsoft.com/en-us/azure/purview/tutorial-using-rest-apis

## REST APIs for collections 

https://learn.microsoft.com/en-us/azure/purview/tutorial-metadata-policy-collections-apis 


## Catalog Data Plan APIs

https://learn.microsoft.com/en-us/rest/api/purview/catalogdataplane/discovery



In [1]:
import urllib.request
import requests
import os 
import json
from pprint import pprint


# load env
from dotenv import load_dotenv
load_dotenv()


from utils.purview import azure_auth, search_catalog, get_asset_byguid

In [2]:
import logging 
logger = logging.getLogger(__name__)

# # write logs to file 
# logging.basicConfig(
#     filename='purview1.log', 
#     format='%(asctime)s %(levelname)s %(name)s %(message)s')


In [3]:
clientid=os.environ.get('app_clientid')
clientsecret=os.environ.get('app_clientsecret')


tokenurl = os.environ.get('azureauth_tokenurl', "https://login.microsoftonline.com/{{tenantid}}/oauth2/v2.0/token")
perviewaccount = os.environ.get('purviewaccountname', "purviewaccount")
purviewendpoint = f"https://{perviewaccount}.purview.azure.com"
purview_url = f"{purviewendpoint}/catalog/api/search/query?api-version=2022-08-01-preview"




## Authenticate and get bearer token

In [4]:
access_token = azure_auth(clientid, clientsecret, tokenurl)

## Search data catalog

In [5]:
res_json = search_catalog(access_token=access_token, purview_url=purview_url)


for i, item in enumerate(res_json['value']): 
    print(f"{i}: {item['name']} - {item.get('entityType')} - {item['@search.score']}")
    pprint(f"\t{item.get('userDescription')}")


0: test_data.csv - azure_datalake_gen2_path - 372.44748
('\tThe Customer Purchase History data asset provides comprehensive and '
 'detailed records of customer transactions and purchase behavior within an '
 'organization. It encompasses a wide range of information, including product '
 'purchases, dates of purchase, quantities, prices, payment methods, and any '
 'associated discounts or promotions.')
1: CustomerProductInsight - azure_synapse_dedicated_sql_table - 319.89362
'\tNone'
2: RawTransaction.csv - azure_datalake_gen2_path - 299.65378
'\tNone'
3: SalesLTSalesOrderHeader.txt - azure_blob_path - 293.8138
'\tNone'
4: SalesOrderHeader - azure_sql_table - 287.142
'\tNone'


In [6]:
res_json = search_catalog(
    keywords='discounts promotions cupcakes',
    access_token=access_token, 
    purview_url=purview_url)

for i, item in enumerate(res_json['value']): 
    print(f"{i}: {item['name']} - {item.get('entityType')} - {item['@search.score']}")
    print(f"\t{item.get('userDescription')}")


0: test_data.csv - azure_datalake_gen2_path - 477.87497
	The Customer Purchase History data asset provides comprehensive and detailed records of customer transactions and purchase behavior within an organization. It encompasses a wide range of information, including product purchases, dates of purchase, quantities, prices, payment methods, and any associated discounts or promotions.


In [11]:
# pprint(res_json['value'][8])

i = 0
item = res_json['value'][i]
pprint(type(item))

print(f"{i}: {item['name']} - {item.get('entityType')} - {item['@search.score']}")

pprint(item)
pprint(item.get('id'))

<class 'dict'>
0: test_data.csv - azure_datalake_gen2_path - 477.87497
{'@search.highlights': {'__hitKeywords': ['promotions', 'discounts'],
                        'description': ['including product purchases, dates of '
                                        'purchase, quantities, prices, payment '
                                        'methods, and any associated '
                                        '<em>discounts</em>',
                                        'or <em>promotions</em>.']},
 '@search.score': 477.87497,
 'assetType': ['Azure Data Lake Storage Gen2'],
 'collectionId': '4hflys',
 'createBy': 'ServiceAdmin',
 'createTime': 1638939694595,
 'entityType': 'azure_datalake_gen2_path',
 'id': '49e0be06-4219-4b93-8a9d-0608977526f7',
 'isIndexed': True,
 'label': ['Microsoft.Label.9FBDE396_1A24_4C79_8EDF_9254A0F35055'],
 'name': 'test_data.csv',
 'objectType': 'Files',
 'owner': '$superuser',
 'qualifiedName': 'https://contosolakeamjad1984.dfs.core.windows.net/user/amldem

## Get item by Guid

In [13]:
res_json = get_asset_byguid(
    itemid=item.get('id'),
    access_token=access_token,
    purviewendpoint=purviewendpoint)

In [14]:
res_json['entity'].keys()

for k, v in res_json['entity'].items():
    print(f"{k}: {v}")

typeName: azure_datalake_gen2_path
attributes: {'owner': '$superuser', 'modifiedTime': 1615254315000, 'replicatedTo': None, 'userDescription': '<div>The Customer Purchase History data asset provides comprehensive and detailed records of customer transactions and purchase behavior within an organization. It encompasses a wide range of information, including product purchases, dates of purchase, quantities, prices, payment methods, and any associated discounts or promotions.<br></div>', 'isFile': True, 'replicatedFrom': None, 'qualifiedName': 'https://contosolakeamjad1984.dfs.core.windows.net/user/amldemo/test_data.csv', 'displayName': None, 'description': None, 'groups': '$superuser', 'ACL': None, 'path': '/user/amldemo/test_data.csv', 'size': 78528, 'name': 'test_data.csv', 'userProperties': None, 'contentType': 'application/octet-stream'}
lastModifiedTS: 8
guid: 49e0be06-4219-4b93-8a9d-0608977526f7
isIncomplete: False
status: ACTIVE
createdBy: ServiceAdmin
updatedBy: efd036e9-c0e2-435